# Understanding Joins

A join operation allows you to retrieve data from multiple tables in a single
SELECT query. 

Two tables can be joined by a single join operator, but the
result can be joined again with other tables.

There must exist a same or
similar column between the tables being joined.

we split our programming
languages table into two in Chapter 7 – one holding the author details
(Table 11-2) and the other holding information about the languages itself

We choose our join column as the language_id field from the authors table.
This corresponds to the id field in the languages table.

```
tesdb=# SELECT author, language
        FROM authors_tbl, newlang_tbl
        WHERE language_id = id;

tesdb=# SELECT author, language
        FROM authors_tbl, newlang_tbl
        WHERE id = language_id;
```

The language_id = id is called the join condition. 

the operator used in the join condition is an equality operator (=), this join
is called an equijoin. 

Another important thing to note is that the columns
participating in the join condition are not the ones we choose to be in the
result of the query.

# Alternative Join Syntax

The SQL-92
standard introduced the JOIN keyword

```
tesdb=# SELECT author, language
        FROM authors_tbl JOIN newlang_tbl
        ON language_id = id;

tesdb=# SELECT author, language
        FROM authors_tbl JOIN newlang_tbl
        ON id = language_id;        

tesdb=# SELECT author, language
        FROM newlang_tbl JOIN authors_tbl
        ON id = language_id;          
```

instead separating the two tables using a comma (thereby
making it a list), we use the JOIN keyword. 

 columns that participate in
the join condition are preceded by the ON keyword. 

The WHERE clause can
then be used after the join condition specification (ON clause) to specify
any further conditions if needed.

A JOIN is equal to a INNER JOIN

all rows that don’t match the join condition
exactly are eliminated are called inner joins. 

![](images/inner_join.png)

Specifying INNER JOIN explicitly


```
tesdb=# SELECT author, language
        FROM authors_tbl INNER JOIN newlang_tbl
        ON language_id = id;
```

# Resolving Ambiguity in Join Columns

what if in our languages table we kept the key field's
name as language_id? This would create an ambiguity in the join
condition, which would become the confusing language_id = language_
id. 

```
tesdb=# CREATE TABLE languages_tbl
        (language_id INTEGER, language VARCHAR(20));
        
        INSERT INTO languages_tbl VALUES (4, 'Tcl');

tesdb=# SELECT author, language
        FROM authors_tbl JOIN languages_tbl
        ON language_id = language_id;    

tesdb=# SELECT author, language
        FROM authors_tbl JOIN languages_tbl
        ON authors_tbl.language_id = languages_tbl.language_id;        
```

Another way to solve such ambiguity is to qualify the columns using
table aliases. 

```
tesdb=# SELECT author, language
        FROM authors_tbl a JOIN newlang_tbl l
        ON a.language_id = l.id;
```

# Outer Joins

 If the rows from the first table that don’t match the
condition are desired in the resultset, we use a left outer join. Otherwise
when rows from the second table are required, we use a right outer join.





![](images/left_join.png)

![](images/right_join.jpg)

Let’s add a single row to the newlang_tbl about the Lisp
programming language, but we will not make any entry into the authors
table for this

```
tesdb=# INSERT INTO newlang_tbl
        VALUES (6, 'Lisp', 1958, 'ANSI');

tesdb=# SELECT language, author
        FROM newlang_tbl n JOIN authors_tbl a
        ON n.id = a.language_id;        

tesdb=# SELECT language, author
        FROM newlang_tbl n LEFT OUTER JOIN authors_tbl a
        ON n.id = a.language_id;
```



convert this into a RIGHT OUTER JOIN

```
tesdb=# SELECT language, author
        FROM authors_tbl a RIGHT OUTER JOIN newlang_tbl n
        ON n.id = a.language_id;
```

Since all right outer joins can be written as left outer joins
(and vice versa), it is rare to find many real-world usages of right outer
joins. 

# Full Joins



![](images/full_join.png)

```

tesdb=# INSERT INTO authors_tbl
        (author_id, author)
        VALUES
        (6, 'Jef');

tesdb=# SELECT language, author
        FROM newlang_tbl n RIGHT OUTER JOIN authors_tbl a 
        ON n.id = a.language_id;        

tesdb=# SELECT language, author
        FROM authors_tbl a FULL JOIN newlang_tbl n
        ON n.id = a.language_id;
```

# Cross Joins

 all possible combinations of rows are made from the tables
participating in the join. 

```
tesdb=# SELECT author, language
        FROM authors_tbl, newlang_tbl;
```

each author and
language combination.

selecting more than one column from the newlang_tbl,

Would the number of combinations increase
dramatically

```
tesdb=# SELECT author, language, year
        FROM authors_tbl CROSS JOIN newlang_tbl;
```

Turns out that no,

I’d advise against running cross joins on actual
production database servers 

# Self Joins

a table within its own columns has meaningful data but
one (or more) of its fields refer to another field in the same table.

for example:
 programming languages
that influenced other programming languages and denote the influence
relationship by the language id

```
tesdb=# CREATE TABLE inflang_tbl (id INTEGER    PRIMARY KEY,
        language VARCHAR(20) NOT NULL,
        influenced_by INTEGER);
        
tesdb=# INSERT INTO inflang_tbl (id, language)
        VALUES (1, 'Fortran');

tesdb=# INSERT INTO inflang_tbl (id, language, influenced_by)
        VALUES (2, 'Pascal', 3);

tesdb=# INSERT INTO inflang_tbl (id, language, influenced_by)
        VALUES (3, 'Algol', 1);

tesdb=# SELECT * FROM inflang_tbl;        

```

Running a self join query


```
tesdb=# SELECT l1.id,
                l1.language,
                l2.language AS influenced
        FROM inflang_tbl l1, inflang_tbl l2
        WHERE l1.id = l2.influenced_by;

tesdb=# SELECT l1.id,
                l1.language,
                l2.language AS influenced
        FROM inflang_tbl l1, inflang_tbl l2
        WHERE l2.id = l1.influenced_by;
                
```

De tweede oplossing is fout. Het is Algol dat Pascal heeft beïnvloed. En niet andersom Pascal dat Algol heeft beïnvloed.

Running a self join query using JOIN

```
tesdb=# SELECT l1.id,
                l1.language,
                l2.language AS influenced
        FROM inflang_tbl l1 JOIN inflang_tbl l2
        ON l1.id = l2.influenced_by;
```

# Non-Equi Joins

```
tesdb=# SELECT id,
                author_id,
                author,
                language
        FROM authors_tbl, newlang_tbl
        WHERE id < author_id;
```

condition that in each row the id field is lesser than the
corresponding author_id field value.


```
tesdb=# SELECT id,
                author_id,
                author,
                language
        FROM authors_tbl, newlang_tbl
        WHERE id < author_id
        AND id = language_id;
```

records where the author of a language has
their author_id value greater than their created languages’ id.